In [ ]:
import os
import time
import glob
import h5py
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
from libra_py import units, data_stat
import libra_py.packages.cp2k.methods as CP2K_methods
from scipy.optimize import curve_fit

# Plot PDOS and energy levels

In [ ]:
fs = 16
plt.rcParams.update({'font.size': fs})
fig = plt.figure(figsize=(3.21*2, 2.41*3))
plt.subplot(1,2,1)
# functionals = ['hse06']
# functionals = ['b3lyp']
fermi_levels = [-4.486700,-3.908853,-4.159659]
c = 0
functional = 'pbe'
lw = 2.0
converged_eigenvalues = np.load(F'{functional}_converged_energies.npy')*units.au2ev
# model_eigenvalues = np.load(F'{functional}_ml_energies_size_{size}.npy')*units.au2ev
plt.plot([0]*len(converged_eigenvalues), ls='-.', lw=lw, color='red')
for i in range(converged_eigenvalues.shape[1]):
    plt.plot(converged_eigenvalues[:,i]-fermi_levels[c], ls='-', lw=lw, color='gray')
for i in range(106,120):
    plt.plot(converged_eigenvalues[:,i]-fermi_levels[c], ls='-', lw=lw, color='blue')
for i in range(120,126):
    plt.plot(converged_eigenvalues[:,i]-fermi_levels[c], ls='-', lw=lw, color='green')
    #plt.plot(model_eigenvalues[:,i]-fermi_levels[c], ls=':', lw=lw)#, color='white')
plt.ylabel('Energy, eV', fontsize=fs)
plt.xlabel('Time, fs', fontsize=fs)
plt.suptitle(F'Si$_{{75}}$H$_{{64}}$ - {functional.upper()}', fontsize=fs)
# plt.xticks(range(0,2001, 500), fontsize=fs)
plt.xticks([400,500,600],fontsize=fs)
plt.yticks(fontsize=fs)
plt.xlim(400, 600)
plt.ylim(-4,3)
plt.subplot(1,2,2)
params = {"path_to_all_pdos": '../1_data_generation/2_si75h64/3_converged_wfn', 
          #"path_to_all_pdos": '../1_data_generation/2_si75h64/4_converged_wfn_b3lyp', 
          #"path_to_all_pdos": '../1_data_generation/2_si75h64/6_converged_wfn_hse06', 
          "atoms": [[1,2] , ['Si','H']],
          "orbitals_cols": [[3], range(4,7), range(7,12), range(12,19)], "orbitals":  ['s','p','d','f'],
          "npoints": 2000, "sigma": 0.07, "shift": 2.0}
fermi_level = fermi_levels[0]
ave_energy_grid, homo_energy, ave_pdos_convolved, pdos_labels, ave_pdos_convolved_total = CP2K_methods.pdos(params)
pdos_label = 'Si' #pdos_labels[i]
for i in range(converged_eigenvalues.shape[1]):
    en = converged_eigenvalues[0,i]-fermi_levels[c]
    plt.plot([0,25], [en, en], lw=lw, color='gray')
for i in range(106,120):
    en = converged_eigenvalues[0,i]-fermi_levels[c]
    plt.plot([0,25], [en, en], lw=lw, color='blue')
for i in range(120,126):
    en = converged_eigenvalues[0,i]-fermi_levels[c]
    plt.plot([0,25], [en, en], lw=lw, color='green')
plt.plot([0, 25],[0,0], color='red', ls='-.', lw=lw)
plt.plot(np.sum(ave_pdos_convolved, axis=0), ave_energy_grid-fermi_level-0.0, label=pdos_label, lw=lw, color='black')
# plt.legend(loc='upper right')
plt.ylim(-4,3)
# plt.suptitle('C$_{60}$ - PBE')
plt.xlabel('pDOS, 1/eV')
# plt.ylabel('Energy, eV')
plt.xticks([])
# plt.yticks([])
plt.tight_layout()
plt.savefig(F'energy_pdos_{functional}.jpg', dpi=600)

# ML model analysis
## Computing the mean absolute error

In [ ]:
def mean_absolute_error(x1,x2):
    diff = x1-x2
    return np.average(np.abs(diff))

In [ ]:
total_energies = []
functionals = ['pbe','b3lyp','hse06']
for functional in functionals:
    print(functional)
    functional_energies = []
    for step in range(999,3001):
        #print(step)
        tmp = []
        for size in [50,100,250,500,750,1000] :
            x = CP2K_methods.read_energies_log_file(F'ml_{functional}_wfns/{functional}_size_{size}_step_{step}.log')
            tmp.append(x)
        functional_energies.append(tmp)
    total_energies.append(functional_energies)
total_energies = np.array(total_energies)
print(total_energies.shape)

In [ ]:
total_energies_reference = []
functional_paths = ['../1_data_generation/2_si75h64/3_converged_wfn',
                    '../1_data_generation/2_si75h64/4_converged_wfn_b3lyp',
                    '../1_data_generation/2_si75h64/6_converged_wfn_hse06']
for functional_path in functional_paths:
    print(functional_path)
    functional_energies = []
    for step in range(999,3001):
        x = CP2K_methods.read_energies_log_file(F'{functional_path}/si75h64_{step}.log')
        functional_energies.append(x)
    total_energies_reference.append(functional_energies)
total_energies_reference = np.array(total_energies_reference)
print(total_energies_reference.shape)

In [ ]:
init_total_energies = []
functional_paths = ['../1_data_generation/2_si75h64/2_atomic_wfn']
for functional_path in functional_paths:
    print(functional_path)
    functional_energies = []
    for step in range(999,3001):
        x = CP2K_methods.read_energies_log_file(F'{functional_path}/si75h64_{step}.log')
        functional_energies.append(x)
    init_total_energies.append(functional_energies)
init_total_energies = np.array(init_total_energies)
print(init_total_energies.shape)

In [ ]:
shuffled_indices = np.load('shuffled_indices/shuffled_indices.npy')

In [ ]:
plt.figure(figsize=(3.21*2, 2.41*2))
fs = 20
plt.rcParams.update({'font.size': fs})
sizes = [50,100,250,500,750,1000]
functionals = ['hse06']
colors = ['blue', 'red', 'green']

for i in [2]:#range(total_energies.shape[0]):
    errors = []
    error = mean_absolute_error(total_energies_reference[i,:,0], init_total_energies[0,:,0])
#     plt.plot(sizes,[error]*len(sizes), color=colors[0], lw=3.0, ls='dashdot')#, label=F'Atomic guess')
    for j in range(total_energies.shape[2]):
        test_indices = shuffled_indices[sizes[j]:]
        error = mean_absolute_error(total_energies[i,test_indices,j,0], total_energies_reference[i,test_indices,0])
        errors.append(error)
    plt.plot(sizes, errors, label=F'Total energy, Ha', marker='s', lw=3.0, markersize=8, color=colors[0])
    
for c, functional in enumerate(['hse06']):
    #plt.subplot(1,2,c+1)
    errors = []
    atomic_eigenvalues = np.load(F'{functional}_atomic_eigenvalues.npy')
    converged_eigenvalues = np.load(F'{functional}_converged_energies.npy')
    atomic_error = mean_absolute_error( atomic_eigenvalues , converged_eigenvalues ) * units.au2ev * 1000 # meV
    for size in sizes:
        print(size)
        test_indices = shuffled_indices[size:]
        model_eigenvalues = np.load(F'{functional}_ml_energies_size_{size}.npy')
        gap_converged = (converged_eigenvalues[:,182] - converged_eigenvalues[:,181])*units.au2ev
        gap_model = (model_eigenvalues[:,182] - model_eigenvalues[:,181])*units.au2ev
        print(np.average(np.abs(gap_converged-gap_model))*1000)
        error = mean_absolute_error(converged_eigenvalues[test_indices,:], model_eigenvalues[test_indices,:])# * units.au2ev * 1000# eV
        errors.append(error)
    plt.plot(sizes, errors, marker='s', markersize=8, label=F'MO energies, Ha', lw=3.0, color=colors[1])
    
for c, functional in enumerate(['hse06']):
    atomic_overlap = np.abs(np.load(F'{functional}_atomic_and_converged_overlap.npy'))
    atomic_error = np.average(1-atomic_overlap)
    print(atomic_error)
    tmp = []
    tmp1 = []
    for size in sizes:
        print(functional, size)
        test_indices = shuffled_indices[size:]
        test_indices = np.delete(test_indices, np.where(test_indices == 2001))
        ml_overlap = np.abs(np.load(F'{functional}_ml_and_converged_overlap_size_{size}.npy'))[test_indices,:]
        ml_error = np.average(1-ml_overlap)
        ml_error_bar = np.std(1-ml_overlap)*1.96/np.sqrt(np.size(ml_overlap))
        print(functional, size, ml_error, '+-', ml_error_bar)#, np.std(1-ml_overlap))
        tmp.append(ml_error)
        tmp1.append(ml_error_bar)
    plt.plot(sizes, tmp, marker='s', markersize=8, lw=3.0, color=colors[2],
             label='$\\epsilon $')
plt.ylim(1.0e-4,1.0)
plt.ylabel('MAE', fontsize=fs)
plt.xlabel('Training set size', fontsize=fs)
plt.xticks(fontsize=fs)
plt.yticks(fontsize=fs)
plt.legend(fontsize=fs-6, loc='upper right')
plt.title('C$_{60}$ - HSE06')
plt.yscale('log')
plt.xscale('log')
plt.yticks([1.0e-4,1.0e-3,1.0e-2,1.0e-1,1.0e0], fontsize=fs)
plt.tight_layout()
plt.savefig(F'error_analysis_{functionals[0]}_test_hartree.jpg', dpi=600)

# Computing timescales from NA-MD calculations

In [ ]:
all_energies_converged = []
all_energies_ml = []
functionals = ['pbe','b3lyp','hse06']
sizes = [50,100,250,500,750,1000]
istep = 0
fstep = 1998
t1 = time.time()
for functional in functionals:
    print(functional)
    size = 250
    energies = []
    for i in range(istep , fstep):
        energy = np.diag(sp.load_npz(F'namd/{functional}_converged/Hvib_sd_{i}_re.npz').todense()).real
        energies.append(energy)
    energies = np.array(energies)*units.au2ev
    all_energies_converged.append(energies)
for functional in functionals:
    energies_functional = []
    for size in sizes:
        print(functional, size)
        energies = []
        for i in range(istep , fstep):
            energy = np.diag(sp.load_npz(F'namd/{functional}_size_{size}/Hvib_sd_{i}_re.npz').todense()).real
            energies.append(energy)
        energies = np.array(energies)*units.au2ev
        energies_functional.append(energies)
    all_energies_ml.append(energies_functional)
all_energies_converged = np.array(all_energies_converged)
all_energies_ml = np.array(all_energies_ml)
print(all_energies_converged.shape, all_energies_ml.shape)
print(time.time()-t1)

In [ ]:
all_e2 = []
all_e2_ml = []
for i in range(3):
    energies = all_energies_converged[i]
    e2 = (energies.T-energies[:,1]).T
    all_e2.append(e2)
for i in range(3):
    tmp = []
    for j in range(6):
        energies = all_energies_ml[i][j]
        e2 = (energies.T-energies[:,1]).T
        tmp.append(e2)
    all_e2_ml.append(tmp)
all_e2 = np.array(all_e2)
all_e2_ml = np.array(all_e2_ml)
print(all_e2.shape, all_e2_ml.shape)

In [ ]:
def exp_func(t, e0, tau, beta):
    return e0*np.exp(-np.power(t/tau, beta))
def double_exp_func(t, e0, a, tau1, tau2):
    return (e0-a)*np.exp(-t/tau1) + a*np.exp(-t/tau2)
def double_exp_func_1(t, e0, a, tau1, tau2, beta1, beta2):
    return (e0-a)*np.exp(-np.power(t/tau1, beta1)) + a*np.exp(-np.power(t/tau2, beta2))
def error_bar(data):
    ave = np.average(data)
    s = np.std(data)
    Z = 1.96
    N = len(data)#.shape[0]
    error = Z*s/np.sqrt(N)
    return ave, error
def icond_gen(file):
    s = ''
    c = -1
    while 'icond' not in s and c>-100:
        s = file[c:-1]
        c -= 1
    icond = int(s.replace('icond_','').replace('/mem_data.hd',''))
    return icond

In [ ]:
#%matplotlib notebook
colors = ['blue','red','green'] 
# methods = ['IDA', 'MSDM']
methods = ['FSSH', 'IDA', 'MSDM'] 
# functionals = ['pbe','hse06','b3lyp'] 
sizes = [50,100,250,500,750,1000] 
r2 = 0.5
tau_1 = []
bet_1 = []
tau_err_1 = []
bet_err_1 = []
for functional in functionals:
    tau_2 = []
    bet_2 = []
    tau_err_2 = []
    bet_err_2 = []
    for size in [0]:
        tau_3 = []
        bet_3 = []
        tau_err_3 = []
        bet_err_3 = []
        for k, method in enumerate(methods):
            print(F'============================ {functional}, {size}, {method}')
            #plt.figure()
            taus = []
            betas = []
            files = glob.glob(F'namd/converged_{method}_functional_{functional}_*/mem_data.hdf')
            for c, file in enumerate(files):
                icond = icond_gen(file)
                if icond<1010:
                    F = h5py.File(file)
                    sh_pop = np.array(F['sh_pop_adi/data'])
                    time = np.array(F['time/data'])[0:1000]*units.au2fs
                    a = sh_pop.shape[1]
                    tmp = np.sum(np.multiply(sh_pop[0:1000,1:], np.roll(all_e2[k1,0:1000,1:a], icond, axis=0)), axis=1)
                    e0 = tmp[0]
                    F.close()
                    popt, pcov = curve_fit( exp_func, time, tmp, bounds=([e0-0.000001, 0.0, 0.5],[e0+0.000001, np.inf, 4.0]))
                    e0, tau, beta = popt
                    # Computing the R-squared
                    residuals  = tmp - exp_func(time, *popt)
                    ss_res     = np.sum(residuals**2)
                    ss_tot     = np.sum((tmp - np.mean(tmp))**2)
                    r_squared  = 1.0 - (ss_res / ss_tot)
                    #print('tau:', tau,  ' R2:', r_squared)

                if r_squared>r2:
                    #line = plt.plot(time, exp_func(time, *popt), ls='dotted')
                    #plt.plot(time, tmp, ls='-', color=line[0].get_color())
                    taus.append(tau)
                    betas.append(beta)
            tau_ave, tau_error = error_bar(taus)
            beta_ave, beta_error = error_bar(betas)
            tau_3.append(tau_ave)
            tau_err_3.append(tau_error)
            bet_3.append(beta_ave)
            bet_err_3.append(beta_error)

            print('Converged: ', functional, method, 'time: ',tau_ave, '+-', tau_error, 'beta: ', beta_ave, '+-', beta_error)
        tau_2.append(tau_3)
        tau_err_2.append(tau_err_3)
        bet_2.append(bet_3)
        bet_err_2.append(bet_err_3)
    tau_1.append(tau_2)
    tau_err_1.append(tau_err_2)
    bet_1.append(bet_2)
    bet_err_1.append(bet_err_2)

conv_tau_1 = np.array(tau_1)
conv_tau_err_1 = np.array(tau_err_1)
conv_bet_1 = np.array(bet_1)
conv_bet_err_1 = np.array(bet_err_1)
print(conv_tau_1.shape, conv_tau_err_1.shape)
print(conv_bet_1.shape, conv_bet_err_1.shape)

In [ ]:
%matplotlib notebook
fs=20
plt.rcParams.update({'font.size': fs})
for i in range(len(functionals)):
    plt.figure(figsize=(3.21*2,2.41*2))
    for j in [0,1,2]:#range(len(methods)):
        plt.plot(sizes, [conv_tau_1[i,0,j]+conv_tau_err_1[i,0,j]]*len(sizes),lw=1.0, 
                            ls='dashdot', color=colors[j])#, marker='s', markersize=8)
        plt.plot(sizes, [conv_tau_1[i,0,j]-conv_tau_err_1[i,0,j]]*len(sizes),lw=1.0, 
                            ls='dashdot', color=colors[j])#, marker='s', markersize=8)
        plt.fill_between(sizes, [conv_tau_1[i,0,j]+conv_tau_err_1[i,0,j]]*len(sizes), 
                         [conv_tau_1[i,0,j]-conv_tau_err_1[i,0,j]]*len(sizes), color=colors[j], alpha=0.5)
        plt.errorbar(sizes, ml_tau_1[i,:,j], yerr=ml_tau_err_1[i,:,j], capsize=4, lw=3.0, 
                     label=F'ML, {methods[j]}', ls='-', color=colors[j], marker='s', markersize=8)
    plt.title(F'Si$_{{75}}$H$_{{64}}$ - {functionals[i].upper()}')
    plt.ylabel('$\\tau$, fs')
    plt.xlabel('Training set size')
    plt.yscale('log')
    plt.xscale('log')
    plt.yticks([7.0e1,1.0e3,4.0e3],["$7\\times10^1$","$10^3$","$4\\times 10^3$"])
    plt.legend(fontsize=fs-3, bbox_to_anchor=(0.55,0.6))
    plt.tight_layout()
    plt.savefig(F'timescales_{functionals[i]}.jpg', dpi=600)